# 4. Few Shot Prompting

## Setup

In [1]:
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

assert os.environ["LANGSMITH_TRACING"] is not None
assert os.environ["LANGSMITH_API_KEY"] is not None
assert os.environ["LANGSMITH_PROJECT"] is not None
assert os.environ["OPENAI_API_KEY"] is not None

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

## 3.1 Introduction

- Few-shot prompting is a technique used with large language models (LLMs) where you provide a small number of examples (typically 1–5) within the prompt to show the model how to perform a task
- Importantly, the examples should include "negatives" so that the LLM understands how to handle such cases.
- Few Shot Prompting typically involvces alternating user and assistant messages that demonstrate the desired behaviour before providing the actual prompt.

In [3]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = model.invoke(messages)
print(response.content)

7


## 3.2 Few Shot Prompting with Tool Calling

- We can use Few Shot Prompting to "teach" an LLM to call a tool correctly. For example: we can "Teach" an LLM to fill a schema. 

In [4]:
from pydantic import BaseModel, Field
from typing import Optional, List

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

**For Few-Shot Prompting, Different LLM providers expect different message sequences**. Typically this starts with:

1. User message
2. AI message with tool call
3. Tool message with result
4. (Optional) A final AI message containing some sort of response.

- LangChain includes a utility function `tool_example_to_messages` that will generate a valid sequence for LLM providers.
- Developers wioll just need to provide the user message and the pydantic output as shown in the example below:

In [5]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (508f6ca7-2d0d-4fec-be0a-fb118d2d0a10)
 Call ID: 508f6ca7-2d0d-4fec-be0a-fb118d2d0a10
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (e8628411-b9b9-4e86-9906-2c482a2a81c2)
 Call ID: e8628411-b9b9-4e86-9906-2c482a2a81c2
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

/var/folders/x1/zrl57ryx10l614s07glvwcnh0000gp/T/ipykernel_85844/3367841236.py:23: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


Notice how we only provided the user message and the expected Pydantic output, and `tool_example_to_messages` converted this to the correct sequence of Human, Ai, Tool, Ai


```
┌────────────┐
│  Human     │ "The ocean is vast and blue..."
└─────┬──────┘
      │
      ▼
┌────────────┐
│   AI       │ Calls tool `Data` with args:
│            │   people: []
└─────┬──────┘
      │
      ▼
┌────────────┐
│   Tool     │ Confirms tool call executed
└─────┬──────┘
      │
      ▼
┌────────────┐
│   AI       │ "Detected no people."
└────────────┘

```


```
┌────────────┐
│  Human     │ "Fiona traveled far from France to Spain."
└─────┬──────┘
      │
      ▼
┌────────────┐
│   AI       │ Calls tool `Data` with args:
│            │   people: [{ name: "Fiona", height: None, hair: None }]
└─────┬──────┘
      │
      ▼
┌────────────┐
│   Tool     │ Confirms tool call executed
└─────┬──────┘
      │
      ▼
┌────────────┐
│   AI       │ "Detected people."
└────────────┘

```

When invoking the LLM, these training examples are placed before the actual question. The model uses them as guidance and then produces its final output

In [6]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = model.with_structured_output(schema=Data)

In [7]:
structured_llm.invoke([message_no_extraction])  # without few-shot prompting

Data(people=[])

In [8]:
structured_llm.invoke(messages+[message_no_extraction]) # with few-shot prompting

Data(people=[])

## 3.5 Playground

In [9]:
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import tool_example_to_messages

class BudgetEntry(BaseModel):
    amount: Optional[float] = Field(description = "The income or expense amount",default=0.0)
    currency: Optional[str] = Field(description = "The currency of the amount",default='AED')
    creditOrDebit: Optional[str] = Field(description = "Credit or Debit. Debit if the amount was debited/spent. credit if the amount was received. Defaults to credit", enum=["C","D"],default='D')
    memo: Optional[str] = Field(description="Short description of the credit/debit event e.g. Shopping")
    category: str = Field(description="The category of the credit/debit event e.g. Bills", enum=["Salary","Bills","Rent","Shopping","Car","Home"])

class Extract(BaseModel):
    entry:  Optional[BudgetEntry] = Field(description = "The budget entry if all of required the details of the transaction were present in the text"),
    success: bool = Field(description="True/False value indicating if the text contained all required details for a transaction")


examples = [
    (
        "Fifty dollars for a t-shirt",
        Extract(success=True, entry=BudgetEntry(amount=50., currency="USD",creditOrDebit="D",memo="T-Shirt",category="Shopping")),
    ),
    (
        "And having the same one as six other people in this club is a hella don't",
        Extract(success=False, entry=None),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.success:
        # This final message is optional for some providers
        ai_response = "Detected entry."
    else:
        ai_response = "Detected no entry."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

In [10]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

Fifty dollars for a t-shirt
================================== Ai Message ==================================
Tool Calls:
  Extract (1e2eed8c-ad7e-4107-af4e-3e334e406f7d)
 Call ID: 1e2eed8c-ad7e-4107-af4e-3e334e406f7d
  Args:
    entry: {'amount': 50.0, 'currency': 'USD', 'creditOrDebit': 'D', 'memo': 'T-Shirt', 'category': 'Shopping'}
    success: True
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected entry.
================================ Human Message =================================

And having the same one as six other people in this club is a hella don't
================================== Ai Message ==================================
Tool Calls:
  Extract (60ab783b-c427-42c6-9777-3411cc16db1f)
 Call ID: 60ab783b-c427-42c6-9777-3411cc16db1f
 

In [11]:
message_with_extraction = {
    "role": "user",
    "content": "Apple Vision Pro thingy for $3999",
}

# Add your few-shot examples + a new query
response = model.invoke(messages + [message_with_extraction])

structured_llm = model.with_structured_output(schema=Extract)
structured_llm.invoke(messages + [message_with_extraction])

/Users/W.Sheikh/Developer/Langchain/.venv/lib/python3.13/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The budget entry if all of required the details of the transaction were present in the text'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/W.Sheikh/Developer/Langchain/.venv/lib/python3.13/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The budget entry if all of required the details of the transaction were present in the text'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


Extract(entry=BudgetEntry(amount=3999.0, currency='USD', creditOrDebit='D', memo='Apple Vision Pro', category='Shopping'), success=True)

In [14]:
message_with_extraction = {
    "role": "user",
    "content": "But shit, it was ninety-nine cents",
}

# Add your few-shot examples + a new query
response = model.invoke(messages + [message_with_extraction])

structured_llm = model.with_structured_output(schema=Extract)
structured_llm.invoke(messages + [message_with_extraction])

/Users/W.Sheikh/Developer/Langchain/.venv/lib/python3.13/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The budget entry if all of required the details of the transaction were present in the text'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/W.Sheikh/Developer/Langchain/.venv/lib/python3.13/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The budget entry if all of required the details of the transaction were present in the text'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


Extract(entry=BudgetEntry(amount=0.99, currency='USD', creditOrDebit='D', memo='Unknown purchase', category='Shopping'), success=True)